In [1]:
from selenium import webdriver
options = webdriver.ChromeOptions() 
options.add_argument('--headless') 
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage') 
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)
import urllib.request
from urllib.error import HTTPError
import time
import datetime

In [21]:
def start_crawling(keyword,START_DATE,END_DATE): #크롤링 시작하는 함수입니다. 키워드, 날짜검색 범위 지정해서 링크 함수로 넘겨줍니다.
    origin_URL = 'ttps://www.sfchronicle.com/search/?action=search&searchindex=solr&query='
    URL = origin_URL + keyword + '&sort=date&page='
    start_date = datetime.datetime.strptime(START_DATE,'%Y-%m-%d')
    last_date = datetime.datetime.strptime(END_DATE,'%Y-%m-%d')
    get_link_from_news_title(URL,start_date,last_date)
            
def get_link_from_news_title(hURL,start_date,last_date):#기사링크 가지고오는 함수입니다.
    firstURL = 'h' + hURL # 하이퍼 링크 에서의 & 변환오류때문에 (& -> &amp;) h를 없애고 마지막에 h를 붙임으로써 오류해결했습니다.
    driver = webdriver.Chrome(executable_path=r'C:/chromedriver/chromedriver.exe')
        # 저는 구글코랩이 아닌, 데스크탑에 설치된 주피터노트북을 사용하였기에, 크롬드라이버.exe파일의 위치를 지정해주어야합니다.
        # 이전에 주신 코드에서 에러가 났었는데 이렇게 쓰면 오류가 나지 않았습니다.
        
    driver.implicitly_wait(7) # 시간 늘렸습니다. (렉 있어서)
    driver.get(firstURL) # 첫화면에서 전체 기사 갯수 가져오기 + 페이지수 계산하기위해 첫화면 들어가줍니다.

    firstsoup = BeautifulSoup(driver.page_source, 'html.parser') 
    num1 = firstsoup.select('p.gsa-message')[0]
    num2 = num1.find_all("strong")[0]
    article_num = int(num2.get_text()) # 날짜계산전 총 기사갯수 크롤링(페이지 수 계산용)
    print(f'날짜 계산 전 총 {article_num}개의 기사 숫자') # 기사 갯수 print 해줍니다
    page_num = int(article_num / 12 ) + 1 # 페이지갯수 계산(1페이지 12개)
    print(f'날짜 계산 전 총 {page_num}개의 페이지 숫자') # 페이지숫자 print 해줍니다.
    
    Flag = True # 시간단축을 위해 Flag = True 설정해줍니다
    for i in range(1,page_num+1): # 페이지 수만큼 반복해줘야합니다. 편의상 1 부터 pagenum + 1 로 range했습니다.
        URL = firstURL + str(i) # URL위에 page num 넣고
        driver.get(URL)           
        soup = BeautifulSoup(driver.page_source, 'html.parser') # soup 따줍니다
        global afterpagenum # 전역변수 설정, page넘어간 숫자 계산

        for i in range(0,len(soup.select('h2.headline'))): 
            title = soup.select('h2.headline')[i]   # i번째의 숫자를 가지고 올수 있게 i , [i] 사용하였습니다.

            title_date1 = soup.select('p.gsa-item-meta')[i] 
            title_date2 = title_date1.select('span.timestamp')[0]
            title_date = title_date2.get_text()
            date = datetime.datetime.strptime(title_date,'%B %d, %Y') # 날짜설정을 위해 날짜부터 가지고와줍니다.

            if start_date >= date:  # 시작날짜부터 if문을 써주고 열어줍니다. (정렬을 날짜순으로 하였기때문에)
                if date >= last_date: # 마지막날까지 date가 안에 있어야하기에 if문 한번 더써줍니다.
                    str_date = str(date)[:11]

                    title_name = title.select('a')[0].get_text()  #기사 제목
                    lstrip_title_name = title_name.lstrip() # 기사제목앞에 불필요한 띄어쓰기가 많아 제거했습니다.
                    title_link = title.select('a')[0]['href'] # 기사링크
                    article_URL = 'https://www.sfchronicle.com/' + title_link

                    TITLE_OF_ARTICLE.append(lstrip_title_name) # 기사 제목 append
                    DATE_OF_ARTICLE.append(str_date) # 기사 날짜 append
                    LINK_OF_ARTICLE.append(article_URL) #기사 URL  append

                    get_text(i,afterpagenum,article_URL) #기사 본문함수로 넘겨줍니다. i와 afterpagenum은 순전히 여태까지 실행한 갯수 알기위해 넣었습니다.
                else: # 원하는 날짜 이후에는 시간 단축을 위해 Flag = False 설정해줍니다
                    Flag = False 
                    break # 반복문 나가줍니다.
        
        afterpagenum += 1 
        print(f'{afterpagenum}페이지 완료') # 진행상황을 알기위해 n페이지 완료 출력해줍니다.
        
        if Flag == False: # 날짜가 지났다면 더이상 찾을 필요가 없으니(날짜별 정렬) 반복문 나가줍니다.
            break
    

def get_text(i,afterpagenum,article_URL): #본문 가져오는 함수입니다.
    req = urllib.request.Request(article_URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})        
    try:
    # 가끔 404 에러가뜨는데, 이건 기사 본문링크가 없어진게 아니고, 
    # 특정 유료(?) 기사 URL로 들어갈때 팝업(결제창)이 뜨면 에러가 나오는것 같습니다.
    # 신문을 구독하면 에러는 안나올 것 같습니다. 구독하기는 껄그러워서 tryexcept로 걸렀습니다. 죄송합니다.
        source_code_from_url = urllib.request.urlopen(req)
        soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
        string_article = '' 
        for article in soup.select('article'):  
            article1 = article.select('p')
            for article2 in article1:
                string_article += "." # 신문사 특성상, 본문이 떨어져있고, 본문마지막에 "." 가 없는 경우도 있어서, 본문마다 .으로 나누었습니다.
                string_article += article2.get_text() # 기사 본문 가져오고 하나씩 더해줍니다.
        if string_article == '':
            # 가끔 기사본문이 없는 경우가 있는데, 이는 기사가 아니라 '자주묻는질문' 페이지로 넘어가서 그런것같습니다. 
            # ex) https://www.sfchronicle.com//faq/
            # 따라서 "IT'S NOT ARTICLE" 을 append 해주었습니다.
            CONTENT_OF_ARTICLES.append("IT'S NOT ARTICLE")
        CONTENT_OF_ARTICLES.append(string_article) # 완성된 기사 본문을 어펜드 해줍니다
    except HTTPError as ERR: # 예외처리이유는 위에서 설명했습니다.
        if ERR.code == 404: # 404에러가 뜨면
            CONTENT_OF_ARTICLES.append('404 NOT FOUND')# 404에러를 기사본문에 추가해줍니다.            
        pass


In [22]:
TITLE_OF_ARTICLE = []
DATE_OF_ARTICLE = []
CONTENT_OF_ARTICLES = []
LINK_OF_ARTICLE = []
afterpagenum = 0

starttime = time.time() # 시작시간 설정

# 전역변수 초기화해줍니다

START_DATE = "2020-12-31" # 위(최신)부터
END_DATE = "2018-01-01" # 아래(과거)까지
# 날짜는 헷갈릴수 있어서 input 넣지않았습니다. 

keyword = input("키워드를 입력하세요 : ")

start_crawling(keyword,START_DATE, END_DATE)
runtime = time.time() - starttime # runtime 출력

print(f'끝! 걸린시간 = {runtime}초')
print(f'날짜 계산 후 총 {len(TITLE_OF_ARTICLE)}개의 기사 숫자')

키워드를 입력하세요 :  ai marketing


날짜 계산 전 총 93개의 기사 숫자
날짜 계산 전 총 8개의 페이지 숫자
1페이지 완료
2페이지 완료
3페이지 완료
끝! 걸린시간 = 82.62770915031433초
날짜 계산 후 총 35개의 기사 숫자


In [65]:
SF = pd.DataFrame(list(zip(LINK_OF_ARTICLE, TITLE_OF_ARTICLE, CONTENT_OF_ARTICLES, DATE_OF_ARTICLE)), columns =['LINK','Title', 'Content', 'Date'])
df = SF

In [66]:
df['Content'] = df['Content'].str.replace(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)" , '')
df['Content'] = df['Content'].str.replace('[', '')
df['Content'] = df['Content'].str.replace(']', '')
df['Content'] = df['Content'].str.replace("'", "")  
df['Content'] = df['Content'].str.replace('"', '')
df['Content'] = df['Content'].str.replace(',', '')
df['Content'] = df['Content'].str.replace(r'\\n', '') 
df['Content'] = df['Content'].str.replace(r'\\r', '')
df['Content'] = df['Content'].str.replace(r'\\xa0', '')
df['Content'] = df['Content'].str.replace('‘', '')
df['Content'] = df['Content'].str.replace('’', '')
df['Content'] = df['Content'].str.replace('“', '')
df['Content'] = df['Content'].str.replace('”', '')
df['Content'] = df['Content'].str.replace('//', '')
df['Content'] = df['Content'].str.replace('ㆍ', '')
df['Content'] = df['Content'].str.replace('  ', '')
df['Content'] = df['Content'].str.replace('·', '')
df['Content'] = df['Content'].str.replace('…', '')
df['Content'] = df['Content'].str.replace('〈br〉', '')
df['Content'] = df['Content'].str.replace('「', '')
df['Content'] = df['Content'].str.replace('」', '')
df['Content'] = df['Content'].str.replace(r'\\', '')
df['Content'] = df['Content'].str.lstrip()
df['Content'] = df['Content'].str.replace(r'\(([^)]+)\)','')

df['Content'] = df['Content'].str.replace('[.][.]', '.')

df['Title'] = df['Title'].str.replace(r'\[([^)]+)\]','')
df['Title'] = df['Title'].str.replace('‘', '')
df['Title'] = df['Title'].str.replace('’', '')
df['Title'] = df['Title'].str.replace(',', '')
df['Title'] = df['Title'].str.replace('[', '')
df['Title'] = df['Title'].str.replace(']', '')
df['Title'] = df['Title'].str.replace('…', '')
df['Title'] = df['Title'].str.replace('-', '')
df['Title'] = df['Title'].str.replace('·', '')
df['Title'] = df['Title'].str.replace('“', '')
df['Title'] = df['Title'].str.replace('”', '')
df['Title'] = df['Title'].str.replace('"', '')
df['Title'] = df['Title'].str.replace('···', '')
df['Title'] = df['Title'].str.replace("'", '')
df['Title'] = df['Title'].str.replace('?', '')
df['Title'] = df['Title'].str.replace('\n', '')

In [67]:
df.head()

LINK  \
0  https://www.sfchronicle.com//business/article/To-free-doctors-from-computers-far-flung-scribes-15617603.php   
1      https://www.sfchronicle.com//shopping/article/Get-lifetime-access-to-over-3-million-design-15591751.php   
2                          https://www.sfchronicle.com//shopping/article/Deals-round-up-or-Aug-31-15527153.php   
3     https://www.sfchronicle.com//shopping/article/amazon-halo-band-fitness-tracker-subscription-15519866.php   
4                          https://www.sfchronicle.com//shopping/article/Deals-round-up-August-12-15478071.php   

                                                                                       Title  \
0                   To free doctors from computers farflung scribes are taking notes           
1  Get lifetime access to over 3 million design assets and stock images for just $50           
2                                                         Deals round up for Aug. 31           
3                          Amazon just announced its Fitbit killer: Amazon Halo Band           
4                                                           Deals round up August 12           

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [42]:
df.to_csv(f'SF_{keyword}({START_DATE}~{END_DATE}).csv', encoding = 'utf-8-sig', index_label = False, index = False)